In [1]:
import numpy as np 
import pandas as pd 
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, precision_recall_curve, average_precision_score, f1_score, roc_auc_score, plot_precision_recall_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN 
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC, LinearSVC, NuSVC

import xgboost
import lightgbm

import matplotlib.pyplot as plt 
import matplotlib.cm as cm 
import seaborn as sns 

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer, word_tokenize

from collections import defaultdict

from lazypredict.Supervised import LazyClassifier

import imblearn

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import flair 

from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis, filter_unlabeled_dataframe
from snorkel.analysis import get_label_buckets
from snorkel.preprocess import preprocessor 
from snorkel.preprocess.nlp import SpacyPreprocessor
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
from snorkel.utils import probs_to_preds



### Steps we will perform in this exercise:

1. Label a small number of samples (df_ground_truth)
2. Use weak supervision (Generator) to label the unlabelled samples (df_unlabelled)
3. Use a supervised model based on ground truth labels and weak supervised labels to classify the sentiment (Discriminator)

## Weak supervision

To generate pseudolabel, we have 4 common types of labeling functions:

1. Hard-coded heuristics (in our problem, boycott tanishq is an obvious giveaway; however, (reject/don't support) boycott tanishq may be an indicaor of an opposite sentiment). We will try to label based on presence of this phrases in the tweet; otherwise we will abstain from labeling the tweet.
2. Syntactics: Spacy's dependency trees can be a very good starting point for generating some more labels.
3. Distant supervision: textblob, flair, vader sentiment analyzer.
4. External models: other models that can generate some good labels.

In [2]:
df = pd.read_csv('../data/processed/tanishq_data_clean_labelled.csv')
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1)
lemmatizer = WordNetLemmatizer()
tokenizer = TweetTokenizer()
df['clean_tweet_token'] = df['clean_tweet_segmented'].apply(lemmatizer.lemmatize).apply(tokenizer.tokenize)
df['clean_tweet_token'] = df['clean_tweet_token'].str.join(' ')
df = df[['clean_tweet_token', 'sentiment']]


In [3]:
df_ground_truth = df[df['sentiment'].isin([0.0, 4.0])]
df_ground_truth['sentiment'] = df_ground_truth['sentiment'].replace({4.0: 1, 0.0: -1})
df_ground_truth_generator, df_ground_truth_discriminator = train_test_split(df_ground_truth, test_size=0.25, stratify=df_ground_truth['sentiment'])

In [4]:
df_unlabelled = df[df['sentiment'].isin([np.nan, 10.0])]
df_unlabelled.drop('sentiment', axis=1, inplace=True)
del df 

In [5]:
#pd.set_option('display.max_colwidth', 0)
#df_ground_truth[df_ground_truth['sentiment'] == -1.00]

In [6]:
# Labeling function

# Textblob 
@preprocessor(memoize=True)
def textblob_sentiment(x):
    scores = TextBlob(x.clean_tweet_token)
    x.tb_polarity = scores.sentiment.polarity
    x.tb_subjectivity = scores.sentiment.subjectivity
    return x 

@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
    return 1 if x.tb_polarity > 0 else -1

# Vader
@preprocessor(memoize=True)
def vader_sentiment(x):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(x.clean_tweet_token)
    x.vd_polarity = scores['compound']
    return x 

@labeling_function(pre=[vader_sentiment])
def vader_polarity(x):
    return 1 if x.vd_polarity > 0 else -1

# Flair
flair_sent = flair.models.TextClassifier.load('en-sentiment')
@preprocessor(memoize=True)
def flair_sentiment(x):
    s = flair.data.Sentence(x.clean_tweet_token)
    flair_sent.predict(s)
    x.fl_polarity =  s.get_label_names()[0].lower()
    return x 

@labeling_function(pre=[flair_sentiment])
def flair_polarity(x):
    return 1 if x.fl_polarity == 'positive' else -1

# Positive sentiments about Tanishq
search = r"(support tanishq | ek at vam)"

@labeling_function()
def positive_tanishq(x):
    return 1 if re.search(search, x.clean_tweet_token, flags=re.I) else 0

# Negative sentiments about Tanishq
search = r"(boycott tanishq | boycott bollywood | boycott amazon | hindu | offended | local | not respect | tradition | hindus | teach | losing | trust)"

@labeling_function()
def negative_tanishq(x):
    return -1 if re.search(search, x.clean_tweet_token, flags=re.I) else 0




2021-02-22 20:52:48,686 loading file /Users/mamu867/.flair/models/sentiment-en-mix-distillbert_3.1.pt


In [7]:
df_ground_truth_generator

,clean_tweet_token,sentiment
209,boycott tanishq jewelry forever in life i won ...,-1.00
1458,boycott tanishq trends on twitter after the je...,-1.00
1135,boycott amazon oh wow all dirts are showing th...,-1.00
331,you need a new team of pr and a good managemen...,-1.00
1851,boycott tanishq please ask pr of tanishq to ha...,-1.00
...,...,...
2118,this dee wali no instead of tanishq buy j well...,-1.00
2133,i don t think tanishq should sell jewels i don...,-1.00
1442,wait do you think tanishq new eka tv am ad is ...,1.00
1648,who is tanishq to advice hindus as how to cele...,-1.00


In [8]:
count_vec = CountVectorizer(ngram_range=(1, 3))
X_count_vec_gen = count_vec.fit_transform(df_ground_truth_generator['clean_tweet_token'])
X_count_vec_dis = count_vec.transform(df_ground_truth_discriminator['clean_tweet_token'])
y_gen = df_ground_truth_generator['sentiment'].values
y_dis = df_ground_truth_discriminator['sentiment'].values

In [9]:
tfidf = TfidfVectorizer(ngram_range=(1, 3))
X_tfidf_vec_gen = tfidf.fit_transform(df_ground_truth_generator['clean_tweet_token'])
X_tfidf_vec_dis = tfidf.transform(df_ground_truth_discriminator['clean_tweet_token'])

In [10]:
lr = LogisticRegression(random_state=42)
knn = KNN()
dt = DecisionTreeClassifier(random_state=42)
xgb = xgboost.XGBClassifier()
lgb = lightgbm.LGBMClassifier(random_state=42)
lda = LinearDiscriminantAnalysis()
svc = SVC(probability=True, random_state=42)
lin_svc = LinearSVC(random_state=42)

classifiers = {'lr': lr, 'knn': knn, 'dt': dt,  'lda': lda, 'svc': svc, 'xgb': xgb, 'lgb': lgb, 'lin_svc': lin_svc}

for clf in classifiers:
    classifiers[clf].fit(X_count_vec_gen.toarray(), y_gen)

@labeling_function()
def lr_label(x):
    return classifiers['lr'].predict(count_vec.transform(x).toarray())

@labeling_function()
def knn_label(x):
    return classifiers['knn'].predict(count_vec.transform(x).toarray())

@labeling_function()
def dt_label(x):
    return classifiers['dt'].predict(count_vec.transform(x).toarray())

@labeling_function()
def xgb_label(x):
    return classifiers['xgb'].predict(count_vec.transform(x).toarray())

@labeling_function()
def lgb_label(x):
    return classifiers['lgb'].predict(count_vec.transform(x).toarray())

@labeling_function()
def lda_label(x):
    return classifiers['lda'].predict(count_vec.transform(x).toarray())

@labeling_function()
def svc_label(x):
    return classifiers['svc'].predict(count_vec.transform(x).toarray())

@labeling_function()
def lin_svc_label(x):
    return classifiers['lin_svc'].predict(count_vec.transform(x).toarray())


In [11]:
lfs = [textblob_polarity, vader_polarity, flair_polarity, positive_tanishq, negative_tanishq, lr_label, knn_label, dt_label, xgb_label, lgb_label, lda_label, svc_label, lin_svc_label]

applier = PandasLFApplier(lfs)
df_unlabelled = df_unlabelled.sample(1500, random_state=42)
L_train = applier.apply(df_unlabelled)

100%|██████████| 1500/1500 [01:35<00:00, 15.75it/s]


In [12]:
LFAnalysis(L_train, lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
textblob_polarity,0,[1],0.16,0.16,0.04
vader_polarity,1,[1],0.18,0.18,0.05
flair_polarity,2,[1],0.02,0.02,0.01
positive_tanishq,3,"[0, 1]",1.00,0.89,0.48
negative_tanishq,4,[0],0.48,0.48,0.48
lr_label,5,[],0.00,0.00,0.00
knn_label,6,[],0.00,0.00,0.00
dt_label,7,[1],0.01,0.01,0.01
xgb_label,8,[1],0.00,0.00,0.00
lgb_label,9,[1],0.01,0.01,0.01


In [13]:
majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

In [14]:
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, lr=0.001, log_freq=100, seed=123)

In [15]:
L_test = applier.apply(df=df_ground_truth_generator[['clean_tweet_token']])

majority_accuracy = majority_model.score(L=L_test, Y=df_ground_truth_generator['sentiment'], tie_break_policy="random", metrics=['accuracy'])['accuracy']

print(f"\n{'Majority Vote Accuracy:':<25} {majority_accuracy * 100:0.1f}%")

label_model_accuracy = label_model.score(L=L_test,Y=df_ground_truth_generator['sentiment'], tie_break_policy="random", metrics=['accuracy'])['accuracy']

print(f"{'Label Model Accuracy:':<25} {label_model_accuracy * 100:0.1f}%")

100%|██████████| 275/275 [00:24<00:00, 11.03it/s]
Majority Vote Accuracy:   100.0%
Label Model Accuracy:     69.2%



In [16]:
buckets = get_label_buckets(L_train[:, 0], L_train[:, 1], L_train[:, 2])
df_unlabelled.iloc[buckets[(1, 1, 1)]].sample(10)

,clean_tweet_token
4833,from next year every hindu shud advice those p...
5516,do not waste you energy on anti hindus anti hi...
1755,celebrate diwali with this great bomb and give...
2726,the most popular hobbies around the world we w...
3805,so true let s boycott tanishq
4969,tanishq is brand of tata co amp i really respe...
8011,enjoy diwali with crackers it s hindu indian f...
6619,hindus please burst as many crackers as possib...
2173,there is no crackers ban in haryana hours up m...
7082,hey tanishq pr team yesterday we did gold shop...


In [17]:
y_ground_truth_pred = majority_model.predict(L_test)
print(classification_report(df_ground_truth_generator['sentiment'].values, y_ground_truth_pred))

              precision    recall  f1-score   support

        -1.0       1.00      0.02      0.04       262
         0.0       0.00      0.00      0.00         0
         1.0       0.05      1.00      0.10        13

    accuracy                           0.07       275
   macro avg       0.35      0.34      0.05       275
weighted avg       0.96      0.07      0.04       275



In [24]:
X_unlabelled = count_vec.transform(df_unlabelled['clean_tweet_token']).toarray()[preds_train != 0]
y_unlabelled = preds_train[preds_train != 0]
y_unlabelled[y_unlabelled == -1] = 0
y_gen[y_gen == -1] = 0
X_train = np.concatenate([X_count_vec_gen.toarray(), X_unlabelled])
y_train = np.concatenate([y_gen, y_unlabelled])
y_dis[y_dis == -1] = 0

In [25]:
np.unique(y_dis)

array([0., 1.])

In [26]:
def custom_metric(A, B):
    d = pd.DataFrame(classification_report(A, B, digits=2,
                                        output_dict=True)).T
    d = d.to_dict()
    d['roc auc'] = roc_auc_score(A, B)
    return d

In [27]:
clf_cv = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=custom_metric)
models_cv, predictions_cv = clf_cv.fit(X_train, X_count_vec_dis.toarray(), y_train, y_dis)

100%|██████████| 30/30 [03:22<00:00,  6.76s/it]


In [28]:
models_cv

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,custom_metric,Time Taken
Model,,,,,,
LGBMClassifier,0.51,0.74,0.74,0.63,"{'precision': {'0.0': 1.0, '1.0': 0.1, 'accura...",1.65
XGBClassifier,0.48,0.72,0.72,0.59,"{'precision': {'0.0': 1.0, '1.0': 0.0943396226...",11.37
BaggingClassifier,0.40,0.68,0.68,0.52,"{'precision': {'0.0': 1.0, '1.0': 0.0833333333...",5.52
DecisionTreeClassifier,0.52,0.65,0.65,0.64,"{'precision': {'0.0': 0.9777777777777777, '1.0...",2.82
QuadraticDiscriminantAnalysis,0.82,0.62,0.62,0.86,"{'precision': {'0.0': 0.9605263157894737, '1.0...",3.33
ExtraTreesClassifier,0.18,0.57,0.57,0.24,"{'precision': {'0.0': 1.0, '1.0': 0.0625, 'acc...",11.16
LinearDiscriminantAnalysis,0.35,0.56,0.56,0.46,"{'precision': {'0.0': 0.9655172413793104, '1.0...",4.66
SGDClassifier,0.16,0.56,0.56,0.20,"{'precision': {'0.0': 1.0, '1.0': 0.0609756097...",1.65
AdaBoostClassifier,0.34,0.56,0.56,0.45,"{'precision': {'0.0': 0.9642857142857143, '1.0...",12.32


In [29]:
X_unlabelled = tfidf.transform(df_unlabelled['clean_tweet_token']).toarray()[preds_train != 0]
#y_unlabelled = preds_train[preds_train != 0]
#y_unlabelled[y_unlabelled == -1] = 0
#y_gen[y_gen == -1] = 0
X_train = np.concatenate([X_tfidf_vec_gen.toarray(), X_unlabelled])
#y_train = np.concatenate([y_gen, y_unlabelled])

In [30]:
clf_tv = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=custom_metric)
models_tv, predictions_tv = clf_tv.fit(X_train, X_tfidf_vec_dis.toarray(), y_train, y_dis)

100%|██████████| 30/30 [03:19<00:00,  6.65s/it]


In [31]:
models_cv

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,custom_metric,Time Taken
Model,,,,,,
LGBMClassifier,0.51,0.74,0.74,0.63,"{'precision': {'0.0': 1.0, '1.0': 0.1, 'accura...",1.65
XGBClassifier,0.48,0.72,0.72,0.59,"{'precision': {'0.0': 1.0, '1.0': 0.0943396226...",11.37
BaggingClassifier,0.40,0.68,0.68,0.52,"{'precision': {'0.0': 1.0, '1.0': 0.0833333333...",5.52
DecisionTreeClassifier,0.52,0.65,0.65,0.64,"{'precision': {'0.0': 0.9777777777777777, '1.0...",2.82
QuadraticDiscriminantAnalysis,0.82,0.62,0.62,0.86,"{'precision': {'0.0': 0.9605263157894737, '1.0...",3.33
ExtraTreesClassifier,0.18,0.57,0.57,0.24,"{'precision': {'0.0': 1.0, '1.0': 0.0625, 'acc...",11.16
LinearDiscriminantAnalysis,0.35,0.56,0.56,0.46,"{'precision': {'0.0': 0.9655172413793104, '1.0...",4.66
SGDClassifier,0.16,0.56,0.56,0.20,"{'precision': {'0.0': 1.0, '1.0': 0.0609756097...",1.65
AdaBoostClassifier,0.34,0.56,0.56,0.45,"{'precision': {'0.0': 0.9642857142857143, '1.0...",12.32


In [32]:
models_cv.to_html('model_cv_pseudolabeling.html')

In [33]:
models_cv['Precision_0'] = models_cv['custom_metric'].apply(lambda x: x['precision']['0.0'])
models_cv['Precision_1'] = models_cv['custom_metric'].apply(lambda x: x['precision']['1.0'])
models_cv['Recall_0'] = models_cv['custom_metric'].apply(lambda x: x['recall']['0.0'])
models_cv['Recall_1'] = models_cv['custom_metric'].apply(lambda x: x['recall']['1.0'])
models_cv['F1_0'] = models_cv['custom_metric'].apply(lambda x: x['f1-score']['0.0'])
models_cv['F1_1'] = models_cv['custom_metric'].apply(lambda x: x['f1-score']['1.0'])
models_cv['ROC_AUC'] = models_cv['custom_metric'].apply(lambda x: x['roc auc'])
models_cv.drop('custom_metric', axis=1, inplace=True)
models_cv

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken,Precision_0,Precision_1,Recall_0,Recall_1,F1_0,F1_1,ROC_AUC
Model,,,,,,,,,,,,
LGBMClassifier,0.51,0.74,0.74,0.63,1.65,1.00,0.10,0.48,1.00,0.65,0.18,0.74
XGBClassifier,0.48,0.72,0.72,0.59,11.37,1.00,0.09,0.45,1.00,0.62,0.17,0.72
BaggingClassifier,0.40,0.68,0.68,0.52,5.52,1.00,0.08,0.37,1.00,0.54,0.15,0.68
DecisionTreeClassifier,0.52,0.65,0.65,0.64,2.82,0.98,0.09,0.51,0.80,0.67,0.15,0.65
QuadraticDiscriminantAnalysis,0.82,0.62,0.62,0.86,3.33,0.96,0.12,0.84,0.40,0.90,0.19,0.62
ExtraTreesClassifier,0.18,0.57,0.57,0.24,11.16,1.00,0.06,0.14,1.00,0.24,0.12,0.57
LinearDiscriminantAnalysis,0.35,0.56,0.56,0.46,4.66,0.97,0.06,0.32,0.80,0.48,0.12,0.56
SGDClassifier,0.16,0.56,0.56,0.20,1.65,1.00,0.06,0.11,1.00,0.21,0.11,0.56
AdaBoostClassifier,0.34,0.56,0.56,0.45,12.32,0.96,0.06,0.31,0.80,0.47,0.12,0.56


In [34]:
models_cv[['Precision_0', 'Precision_1', 'Recall_0', 'Recall_1', 'F1_0', 'F1_1', 'ROC AUC']].head(5).style.background_gradient(cmap='PuBu')

,Precision_0,Precision_1,Recall_0,Recall_1,F1_0,F1_1,ROC AUC
Model,,,,,,,
LGBMClassifier,1.00,0.10,0.48,1.00,0.65,0.18,0.74
XGBClassifier,1.00,0.09,0.45,1.00,0.62,0.17,0.72
BaggingClassifier,1.00,0.08,0.37,1.00,0.54,0.15,0.68
DecisionTreeClassifier,0.98,0.09,0.51,0.80,0.67,0.15,0.65
QuadraticDiscriminantAnalysis,0.96,0.12,0.84,0.40,0.90,0.19,0.62


In [35]:
models_tv['Precision_0'] = models_tv['custom_metric'].apply(lambda x: x['precision']['0.0'])
models_tv['Precision_1'] = models_tv['custom_metric'].apply(lambda x: x['precision']['1.0'])
models_tv['Recall_0'] = models_tv['custom_metric'].apply(lambda x: x['recall']['0.0'])
models_tv['Recall_1'] = models_tv['custom_metric'].apply(lambda x: x['recall']['1.0'])
models_tv['F1_0'] = models_tv['custom_metric'].apply(lambda x: x['f1-score']['0.0'])
models_tv['F1_1'] = models_tv['custom_metric'].apply(lambda x: x['f1-score']['1.0'])
models_tv['ROC_AUC'] = models_tv['custom_metric'].apply(lambda x: x['roc auc'])
models_tv.drop('custom_metric', axis=1, inplace=True)
models_tv

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken,Precision_0,Precision_1,Recall_0,Recall_1,F1_0,F1_1,ROC_AUC
Model,,,,,,,,,,,,
DecisionTreeClassifier,0.39,0.68,0.68,0.51,1.45,1.00,0.08,0.36,1.00,0.53,0.15,0.68
BaggingClassifier,0.35,0.66,0.66,0.46,3.19,1.00,0.08,0.31,1.00,0.47,0.14,0.66
ExtraTreeClassifier,0.29,0.63,0.63,0.39,0.94,1.00,0.07,0.25,1.00,0.40,0.13,0.63
LGBMClassifier,0.26,0.61,0.61,0.35,1.67,1.00,0.07,0.22,1.00,0.36,0.13,0.61
XGBClassifier,0.26,0.61,0.61,0.35,10.99,1.00,0.07,0.22,1.00,0.36,0.13,0.61
SGDClassifier,0.20,0.57,0.57,0.25,1.34,1.00,0.06,0.15,1.00,0.26,0.12,0.57
ExtraTreesClassifier,0.17,0.56,0.56,0.22,9.66,1.00,0.06,0.13,1.00,0.22,0.12,0.56
PassiveAggressiveClassifier,0.35,0.56,0.56,0.46,1.30,0.97,0.06,0.32,0.80,0.48,0.12,0.56
RandomForestClassifier,0.15,0.55,0.55,0.18,4.04,1.00,0.06,0.10,1.00,0.19,0.11,0.55


In [36]:
models_tv[['Precision_0', 'Precision_1', 'Recall_0', 'Recall_1', 'F1_0', 'F1_1', 'ROC AUC']].head(5).style.background_gradient(cmap='PuBu')

,Precision_0,Precision_1,Recall_0,Recall_1,F1_0,F1_1,ROC AUC
Model,,,,,,,
DecisionTreeClassifier,1.00,0.08,0.36,1.00,0.53,0.15,0.68
BaggingClassifier,1.00,0.08,0.31,1.00,0.47,0.14,0.66
ExtraTreeClassifier,1.00,0.07,0.25,1.00,0.40,0.13,0.63
LGBMClassifier,1.00,0.07,0.22,1.00,0.36,0.13,0.61
XGBClassifier,1.00,0.07,0.22,1.00,0.36,0.13,0.61


In [37]:
models_tv

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken,Precision_0,Precision_1,Recall_0,Recall_1,F1_0,F1_1,ROC_AUC
Model,,,,,,,,,,,,
DecisionTreeClassifier,0.39,0.68,0.68,0.51,1.45,1.00,0.08,0.36,1.00,0.53,0.15,0.68
BaggingClassifier,0.35,0.66,0.66,0.46,3.19,1.00,0.08,0.31,1.00,0.47,0.14,0.66
ExtraTreeClassifier,0.29,0.63,0.63,0.39,0.94,1.00,0.07,0.25,1.00,0.40,0.13,0.63
LGBMClassifier,0.26,0.61,0.61,0.35,1.67,1.00,0.07,0.22,1.00,0.36,0.13,0.61
XGBClassifier,0.26,0.61,0.61,0.35,10.99,1.00,0.07,0.22,1.00,0.36,0.13,0.61
SGDClassifier,0.20,0.57,0.57,0.25,1.34,1.00,0.06,0.15,1.00,0.26,0.12,0.57
ExtraTreesClassifier,0.17,0.56,0.56,0.22,9.66,1.00,0.06,0.13,1.00,0.22,0.12,0.56
PassiveAggressiveClassifier,0.35,0.56,0.56,0.46,1.30,0.97,0.06,0.32,0.80,0.48,0.12,0.56
RandomForestClassifier,0.15,0.55,0.55,0.18,4.04,1.00,0.06,0.10,1.00,0.19,0.11,0.55
